In [4]:
import string
from datetime import datetime
import os
import numpy as np
import pandas as pd
import json
from collections import defaultdict
from Queue import Queue
# import pickle
import cPickle as pickle

from scipy.sparse.linalg import svds

import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer

from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.decomposition import LatentDirichletAllocation, NMF
from sklearn.metrics.pairwise import cosine_similarity

In [5]:
valid_business_ids = set()
business_id_to_index = {}
business_index_to_bid = {}
c = 0
with open("biz_food.json", "r") as f:
    for line in f:
        business = json.loads(line)
        valid_business_ids.add(business["business_id"])
        if business["business_id"] not in business_id_to_index:
            business_id_to_index[business["business_id"]] = c
            business_index_to_bid[c] = business["business_id"]
            c += 1

In [7]:
print(len(valid_business_ids))
# business_ids[:5]

60702


In [35]:
" ".join([])

''

In [6]:
only_text_nouns = []
only_text_adjs = []
ordered_review_ids = []
ordered_business_id_indices = []
bid_to_rid_dict = defaultdict(list)
rid_to_bid_dict = {}

with open("yelp_academic_dataset_review.json", "r") as f:
    for line in f:
        review = json.loads(line)
        if review["business_id"] in valid_business_ids:
#             only_text.append(review["text"])
            text = review["text"].lower()
            tokens = nltk.word_tokenize(text)
            pos_tagged = nltk.pos_tag(tokens)
            nouns = [token for (token, pos) in pos_tagged if pos[0] == "N"]
            adjs = [token for (token, pos) in pos_tagged if pos[0] == "J"]
            only_text_nouns.append(" ".join(nouns))
            only_text_adjs.append(" ".join(adjs))
            ordered_review_ids.append(review["review_id"])
            ordered_business_id_indices.append(business_id_to_index[review["business_id"]])
            bid_to_rid_dict[review["business_id"]].append(review["review_id"])
            rid_to_bid_dict[review["review_id"]] = review["business_id"]
print(len(only_text_nouns))
print(len(only_text_adjs))
print(len(ordered_review_ids))
print(only_text_nouns[:5])
print(only_text_adjs[:5])

2874628
2874628
2874628
[u'place area staple years i thing family year pattys day corn beef place bar night dinner friends drinks', u'mojo teasers lpw bite', u'decor staff pickles value', u'i stars mind type restaurant name ohio maisonette restaurant cincy time type venue folks price lunch saturday lady cheeseburger fries i order pickles app pickles spears amount beer batter perfection spears chips decision lady askew type fare plate share kicker bucks pickle fries amount crispness interior fry guy craft burger well.. genre burger scrod portion lemon butter i option fries potato rice i option broccoli amount share dinner salad bag-o-lettuce romaine slice carrot shreds tomato store dinner roll butter service waitress bit management issue server fault cleveland entertainment book i coupon table meal waitress value ticket efficiency restaurant move star heartbeat', u'food laid place nothing mind people restaurant food prices']
[u'good stable reliable st. nice', u'few appetite no-frills', 

In [7]:
rid_to_index = {}
for ind, rid in enumerate(ordered_review_ids):
    rid_to_index[rid] = ind
rid_to_index

{u'Qo62c0I5ydN_FrUbG0NOow': 221295,
 u'uJMipKiUD_nGl3q9vfEuCw': 1187650,
 u'mMqrghG86-vWiwRYjZ0QPQ': 1902993,
 u'Dxmcgjf9JSBlYmxC4ArzHg': 1813536,
 u'6rLbesEeU-IsnbTurUSs9w': 220806,
 u'gOMOMec8QMWJ72gMQsEjlA': 1095087,
 u'wifZGObIDBp4h6Xlm36T2g': 2230697,
 u'PiKeptZU8R09xi-L3HmqQg': 424632,
 u'-PWR-Q9c7ajqE3T3OBpDLw': 1007067,
 u'RIFfCpf3a8r5Dtf8HFvesg': 921919,
 u'RDcIcfslKtUaX2k9foYDqg': 15476,
 u'8ejjNdl6yeT8v3rnpybo_g': 2375675,
 u'xyeHv3Y5ws6t2Q5Whc7e1g': 1770043,
 u'te2CNInedKYW1a94QyWDZQ': 979302,
 u'z0qi0voFDsabGkj10a4FqQ': 2155175,
 u'gUnS7kWlZDorC0GZaGH7MA': 1152825,
 u'LkWt7SgIb5bAzZBywCrGAA': 1155204,
 u'UMdivlESxH-P5m-3uf9-1g': 2256792,
 u'6Xsike7WkPwdLUwpYmkV6A': 28226,
 u'hyWADYuoJAhEfXgk9OGKPQ': 1819328,
 u'u48IrPCi3-JbqBWVmXCfpg': 968039,
 u'PnENj9Cq4BHSNyUmP_sSQA': 741524,
 u'54ltyfKEtTUmnvjcIFcWxA': 1471406,
 u'J_HQGQ9z4mXllbGBM31N7A': 2233051,
 u'N6_1ESmUfTPVpMCHPyiKBg': 2258764,
 u'Zg1L6tLM8T0hRpX7Wlon2w': 874849,
 u't5J0yCUOE_dF9SLoJg7VYw': 2673881,
 u'bcM_TJD0SP

In [18]:
with open("data_structs.pkl", "wb") as f:
    pickle.dump((only_text_nouns,only_text_adjs,bid_to_rid_dict,rid_to_index), f)

In [22]:
tfidf_vectorizer_nouns = TfidfVectorizer(input=only_text_nouns, lowercase=True, stop_words="english", max_df=0.9, min_df=1000)
tfidf_review_nouns = tfidf_vectorizer_nouns.fit_transform(only_text_nouns)
tfidf_nouns_feature_names = tfidf_vectorizer_nouns.get_feature_names()

print(tfidf_review_nouns.shape)

(2874628, 4756)


In [23]:
count_vectorizer_nouns = CountVectorizer(input=only_text_nouns, lowercase=True, stop_words="english", max_df=0.9, min_df=1000)
count_review_nouns = count_vectorizer_nouns.fit_transform(only_text_nouns)
count_nouns_feature_names = count_vectorizer_nouns.get_feature_names()

print(count_review_nouns.shape)

(2874628, 4756)


In [8]:
tfidf_vectorizer_adjs = TfidfVectorizer(input=only_text_adjs, lowercase=True, stop_words="english", max_df=0.9, min_df=1000)
tfidf_review_adjs = tfidf_vectorizer_adjs.fit_transform(only_text_adjs)
tfidf_adjs_feature_names = tfidf_vectorizer_adjs.get_feature_names()

print(tfidf_review_adjs.shape)

(2874628, 1856)


In [20]:
count_vectorizer_adjs = CountVectorizer(input=only_text_adjs, lowercase=True, stop_words="english", max_df=0.9, min_df=1000)
count_review_adjs = count_vectorizer_adjs.fit_transform(only_text_adjs)
count_adjs_feature_names = count_vectorizer_adjs.get_feature_names()

print(count_review_adjs.shape)

(2874628, 1856)


In [9]:
def display_topics(model, feature_names, n_top_words):
    for topic_index, topic in enumerate(model.components_):
        print("Topic %d:" % topic_index)
        print(" ".join([feature_names[i] for i in topic.argsort()[:-n_top_words:-1]]))

In [24]:
noun_lda_model = LatentDirichletAllocation(n_topics=30).fit(count_review_nouns)
# display_topics(noun_lda_model, count_nouns_feature_names, 20)

/home/amcire/anaconda2/lib/python2.7/site-packages/sklearn/decomposition/online_lda.py:508: DeprecationWarning: The default value for 'learning_method' will be changed from 'online' to 'batch' in the release 0.20. This warning was introduced in 0.18.
  DeprecationWarning)


In [25]:
noun_lda_model_20 = LatentDirichletAllocation(n_topics=20).fit(count_review_nouns)
# display_topics(noun_lda_model, count_nouns_feature_names, 20)

/home/amcire/anaconda2/lib/python2.7/site-packages/sklearn/decomposition/online_lda.py:508: DeprecationWarning: The default value for 'learning_method' will be changed from 'online' to 'batch' in the release 0.20. This warning was introduced in 0.18.
  DeprecationWarning)


In [26]:
noun_lda_model_10 = LatentDirichletAllocation(n_topics=10).fit(count_review_nouns)
# display_topics(noun_lda_model_10, count_nouns_feature_names, 20)

/home/amcire/anaconda2/lib/python2.7/site-packages/sklearn/decomposition/online_lda.py:508: DeprecationWarning: The default value for 'learning_method' will be changed from 'online' to 'batch' in the release 0.20. This warning was introduced in 0.18.
  DeprecationWarning)


In [27]:
noun_nmf_model = NMF(n_components=30).fit(tfidf_review_nouns)
# display_topics(noun_nmf_model, tfidf_nouns_feature_names, 20)

In [28]:
noun_nmf_model_20 = NMF(n_components=20).fit(tfidf_review_nouns)
# display_topics(noun_nmf_model, tfidf_nouns_feature_names, 20)

In [29]:
noun_nmf_model_10 = NMF(n_components=10).fit(tfidf_review_nouns)
# display_topics(noun_nmf_model, tfidf_nouns_feature_names, 20)

In [92]:
display_topics(noun_lda_model, count_nouns_feature_names, 20)

Topic 0:
pizza coffee place crust slice pizzas cheese tastes cup sauce latte pastries slices smell shop pepperoni bagel espresso dough
Topic 1:
vegas strip bbq kind man love hands bartenders tv cheesecake rock dollar buns hole airport way king stay ass
Topic 2:
order time people minutes hour reviews potatoes hours money orders way stars pie place tip delivery guess combo reason
Topic 3:
location breakfast brunch trip bacon toast morning egg pancakes station eat waffles notch day hash caesar yesterday biscuits times
Topic 4:
tea home options size stuff portion half option bite flavor place milk son people thing bit lot sunday way
Topic 5:
salad water meal husband wife sauce plate dinner bread eggs menu cheese boyfriend meat piece appetizer waiter tender sides
Topic 6:
tacos manager party staff rest card head management favorites people donuts credit person goodness alcohol reservations things mistake charge
Topic 7:
store butter counter parking employees girl guy lot lady line hand surp

In [93]:
display_topics(noun_lda_model_20, count_nouns_feature_names, 20)

Topic 0:
lunch dishes salad buffet pho lamb dish dressing buffets dumplings specials sea girlfriend ranch mussels seafood restaurants chipotle price
Topic 1:
selection patio beers job plenty sandwiches salty duck meals wine corner tv locations nights diner hash tap staff bellagio
Topic 2:
sandwich bacon cheese today bread kids brunch lunch onion egg toast pancakes number menu order slices mac turkey daughter
Topic 3:
table waitress restaurant waiter night party group dinner water room experience friends orders menu hotel business reservation birthday people
Topic 4:
bar beer place area staff family fun friends game tables atmosphere lot neighborhood lots room space location sports people
Topic 5:
la care son par cute cooking le super que pas je pour au mon et service sucks des les
Topic 6:
sushi rolls roll crab shrimp fish lobster place seafood variety menu guys salmon quality oysters spring service price sashimi
Topic 7:
coffee breakfast place trip cake counter morning shop staff day 

In [94]:
display_topics(noun_lda_model_10, count_nouns_feature_names, 20)

Topic 0:
place tea flavors time variety orders man date love thing counter bottle flavor trip spice size toppings juice boyfriend
Topic 1:
food service time order table minutes server experience restaurant customer place waitress wait manager people waiter times drinks night
Topic 2:
chicken salad sauce rice soup dish spicy ice shrimp beef thai dishes crab meal flavor order bread butter dessert
Topic 3:
pizza food place buffet tacos chips chicken order salsa crust burrito pho service plenty taco time options ingredients delivery
Topic 4:
store business line people lobster eggs parking lot way time end location employees day card lady things review customers
Topic 5:
coffee breakfast chocolate cake cream shop place brunch cup toast morning la hair cakes dessert desserts waffles cookies bacon
Topic 6:
location town wine place glass week spot problem years area rest level house work staff family toronto selection places
Topic 7:
fries burger cheese sandwich pork meat steak sauce chicken b

In [95]:
display_topics(noun_nmf_model, tfidf_nouns_feature_names, 20)

Topic 0:
server table minutes experience waitress manager meal wait waiter tables water hostess party check hour way servers stars reservation
Topic 1:
pizza crust pizzas toppings slice sauce cheese pepperoni delivery slices pie style sausage dough ingredients pasta york knots mozzarella
Topic 2:
food prices portions atmosphere quality family times restaurants places home ambiance stars servers bland taste decor wait waitress court
Topic 3:
place friends town places family area reviews stars owner prices years try thing kind yelp love times home way
Topic 4:
burger fries burgers bacon bun onion cheese rings potato patty juicy shake joint toppings medium cheeseburger onions meat chili
Topic 5:
bar area bartender drink sports bartenders game music room tables beers bars tv games crowd patio fun tvs pool
Topic 6:
sushi rolls roll fish sashimi tuna ayce salmon tempura chef hour quality places spicy chefs nigiri miso pieces rice
Topic 7:
service customer times stars notch prices ambiance st

In [96]:
display_topics(noun_nmf_model_20, tfidf_nouns_feature_names, 20)

Topic 0:
restaurant menu dinner meal experience steak server table waiter wine night salad dessert family dishes vegas restaurants dish bread
Topic 1:
pizza crust wings pizzas toppings sauce cheese slice pepperoni delivery slices salad pie style sausage pasta ingredients dough york
Topic 2:
food price quality prices portions atmosphere buffet wait family places restaurants times people stars taste money waitress servers lot
Topic 3:
place people friends places vegas town family reviews prices area owner lot thing years stars way kind try yelp
Topic 4:
burger fries burgers cheese bacon bun onion potato rings patty juicy meat shake sandwich medium joint toppings onions sauce
Topic 5:
bar drinks night drink bartender hour music people area friends bartenders fun table room patio tables spot sports crowd
Topic 6:
sushi rolls roll fish quality sashimi tuna price ayce salmon tempura places chef hour vegas chefs spicy buffet prices
Topic 7:
coffee shop starbucks cup latte tea morning pastries

In [97]:
display_topics(noun_nmf_model_10, tfidf_nouns_feature_names, 20)

Topic 0:
chicken sauce salad rice lunch restaurant soup beef menu meat pork meal sandwich dish dishes flavor spicy dinner bread
Topic 1:
pizza crust wings pizzas toppings sauce cheese slice delivery pepperoni slices salad pie style sausage pasta ingredients service dough
Topic 2:
food service restaurant quality price prices customer atmosphere portions experience server stars times family restaurants buffet waitress ambiance vegas
Topic 3:
place people friends places vegas town family prices reviews area lot thing owner years stars way try kind pho
Topic 4:
burger fries burgers cheese bacon bun onion potato rings sandwich patty juicy shake meat toppings joint medium onions cheeseburger
Topic 5:
bar beer drinks night selection drink menu hour bartender music beers area wine people friends patio dinner fun atmosphere
Topic 6:
sushi rolls roll fish quality sashimi tuna price ayce salmon service tempura places buffet chef rice vegas spicy hour
Topic 7:
coffee breakfast cream eggs ice shop 

In [58]:
adj_lda_model = LatentDirichletAllocation(n_topics=30).fit(count_review_adjs)
# display_topics(adj_lda_model, count_adjs_feature_names, 20)

/home/amcire/anaconda2/lib/python2.7/site-packages/sklearn/decomposition/online_lda.py:508: DeprecationWarning: The default value for 'learning_method' will be changed from 'online' to 'batch' in the release 0.20. This warning was introduced in 0.18.
  DeprecationWarning)


Topic 0:
good fresh tasty soft impressed limited white daily biggest sick heavy sized crisp buffet canadian medium cheapest healthier mild
Topic 1:
small extra able unique japanese salad friday fabulous efficient cooked mediocre strange seasoned roasted plentiful adventurous refreshing salted rosemary
Topic 2:
good dry prime close disappointing fair pricey higher difficult decent upscale shrimp complete satisfied modern lower correct bbq proper
Topic 3:
sure happy good korean casual friendly vegan professional important separate courteous menu thursday diet strawberry pineapple select quite make
Topic 4:
best decent wonderful cold beautiful incredible dish fluffy fatty natural taco ice wicked pickled younger darn foie eager gracious
Topic 5:
good pretty cool chinese loud american crazy awful standard classic worse funny honest nasty interested crowded miss closest veggie
Topic 6:
super huge high regular short friendly dark rare sad surprised hawaiian food drive future end polish lean s

In [59]:
adj_lda_model_20 = LatentDirichletAllocation(n_topics=20).fit(count_review_adjs)
# display_topics(adj_lda_model, count_adjs_feature_names, 20)

/home/amcire/anaconda2/lib/python2.7/site-packages/sklearn/decomposition/online_lda.py:508: DeprecationWarning: The default value for 'learning_method' will be changed from 'online' to 'batch' in the release 0.20. This warning was introduced in 0.18.
  DeprecationWarning)


Topic 0:
good fresh tasty soft impressed limited white daily biggest sick heavy sized crisp buffet canadian medium cheapest healthier mild
Topic 1:
small extra able unique japanese salad friday fabulous efficient cooked mediocre strange seasoned roasted plentiful adventurous refreshing salted rosemary
Topic 2:
good dry prime close disappointing fair pricey higher difficult decent upscale shrimp complete satisfied modern lower correct bbq proper
Topic 3:
sure happy good korean casual friendly vegan professional important separate courteous menu thursday diet strawberry pineapple select quite make
Topic 4:
best decent wonderful cold beautiful incredible dish fluffy fatty natural taco ice wicked pickled younger darn foie eager gracious
Topic 5:
good pretty cool chinese loud american crazy awful standard classic worse funny honest nasty interested crowded miss closest veggie
Topic 6:
super huge high regular short friendly dark rare sad surprised hawaiian food drive future end polish lean s

In [60]:
adj_lda_model_10 = LatentDirichletAllocation(n_topics=10).fit(count_review_adjs)
# display_topics(adj_lda_model, count_adjs_feature_names, 20)

/home/amcire/anaconda2/lib/python2.7/site-packages/sklearn/decomposition/online_lda.py:508: DeprecationWarning: The default value for 'learning_method' will be changed from 'online' to 'batch' in the release 0.20. This warning was introduced in 0.18.
  DeprecationWarning)


Topic 0:
good fresh tasty soft impressed limited white daily biggest sick heavy sized crisp buffet canadian medium cheapest healthier mild
Topic 1:
small extra able unique japanese salad friday fabulous efficient cooked mediocre strange seasoned roasted plentiful adventurous refreshing salted rosemary
Topic 2:
good dry prime close disappointing fair pricey higher difficult decent upscale shrimp complete satisfied modern lower correct bbq proper
Topic 3:
sure happy good korean casual friendly vegan professional important separate courteous menu thursday diet strawberry pineapple select quite make
Topic 4:
best decent wonderful cold beautiful incredible dish fluffy fatty natural taco ice wicked pickled younger darn foie eager gracious
Topic 5:
good pretty cool chinese loud american crazy awful standard classic worse funny honest nasty interested crowded miss closest veggie
Topic 6:
super huge high regular short friendly dark rare sad surprised hawaiian food drive future end polish lean s

In [10]:
adj_nmf_model = NMF(n_components=30).fit(tfidf_review_adjs)
# display_topics(adj_nmf_model, tfidf_adjs_feature_names, 20)

In [62]:
adj_nmf_model_20 = NMF(n_components=20).fit(tfidf_review_adjs)
# display_topics(adj_nmf_model, tfidf_adjs_feature_names, 20)

Topic 0:
good overall expensive reasonable fried solid hard ok cheap fish dry cheese garlic high slow cool real loud chicken
Topic 1:
great local wonderful reasonable live cool attentive outstanding overall atmosphere fun wine casual outdoor fabulous beautiful unique comfortable affordable
Topic 2:
small large expensive high tiny local worth hungry overall hard disappointed main limited extra smaller cozy japanese bigger larger
Topic 3:
best long mexican italian las indian worth chinese incredible authentic outstanding real fried korean fish average french entire phenomenal
Topic 4:
nice helpful attentive overall cool outside quiet pretty outdoor comfortable pleasant open beautiful warm local casual main able loud
Topic 5:
delicious authentic attentive wonderful beautiful flavorful healthy unique french glad worth vegetarian garlic yummy reasonable generous affordable warm fried
Topic 6:
friendly helpful attentive fast quick reasonable efficient knowledgeable prompt wait local professi

In [63]:
adj_nmf_model_10 = NMF(n_components=10).fit(tfidf_review_adjs)
# display_topics(adj_nmf_model, tfidf_adjs_feature_names, 20)

Topic 0:
good overall expensive reasonable fried solid hard ok cheap fish dry cheese garlic high slow cool real loud chicken
Topic 1:
great local wonderful reasonable live cool attentive outstanding overall atmosphere fun wine casual outdoor fabulous beautiful unique comfortable affordable
Topic 2:
small large expensive high tiny local worth hungry overall hard disappointed main limited extra smaller cozy japanese bigger larger
Topic 3:
best long mexican italian las indian worth chinese incredible authentic outstanding real fried korean fish average french entire phenomenal
Topic 4:
nice helpful attentive overall cool outside quiet pretty outdoor comfortable pleasant open beautiful warm local casual main able loud
Topic 5:
delicious authentic attentive wonderful beautiful flavorful healthy unique french glad worth vegetarian garlic yummy reasonable generous affordable warm fried
Topic 6:
friendly helpful attentive fast quick reasonable efficient knowledgeable prompt wait local professi

In [98]:
display_topics(adj_lda_model, count_adjs_feature_names, 20)

Topic 0:
good fresh tasty soft impressed limited white daily biggest sick heavy sized crisp buffet canadian medium cheapest healthier mild
Topic 1:
small extra able unique japanese salad friday fabulous efficient cooked mediocre strange seasoned roasted plentiful adventurous refreshing salted rosemary
Topic 2:
good dry prime close disappointing fair pricey higher difficult decent upscale shrimp complete satisfied modern lower correct bbq proper
Topic 3:
sure happy good korean casual friendly vegan professional important separate courteous menu thursday diet strawberry pineapple select quite make
Topic 4:
best decent wonderful cold beautiful incredible dish fluffy fatty natural taco ice wicked pickled younger darn foie eager gracious
Topic 5:
good pretty cool chinese loud american crazy awful standard classic worse funny honest nasty interested crowded miss closest veggie
Topic 6:
super huge high regular short friendly dark rare sad surprised hawaiian food drive future end polish lean s

In [99]:
display_topics(adj_lda_model_20, count_adjs_feature_names, 20)

Topic 0:
clean overall nice friendly attentive beautiful hungry fresh pleasant japanese cute tasty blue prepared cheaper smoked helpful pretty strange
Topic 1:
good bad slow reasonable better big horrible unique sure grilled comfortable salad huge rare personal half right spectacular recent
Topic 2:
great local glad wish wine funny downtown organic big entree unlimited greatest shredded ish ordered adorable tad outdoor canadian
Topic 3:
excellent fresh perfect delicious long wonderful fast garlic outstanding poor korean chicken huge fair lunch consistent sure right ridiculous
Topic 4:
amazing cheap black frozen typical total greek knowledgeable larger cooked various pre major seasoned delightful longer middle tuesday corporate
Topic 5:
favorite old open ok entire pricey gross bad negative positive true pizza sure welcome ny sticky parmesan inedible soy
Topic 6:
available fun vegan las est le une proper plain la vous nous younger pour public tried au flavored mon
Topic 7:
mexican white 

In [100]:
display_topics(adj_lda_model_10, count_adjs_feature_names, 20)

Topic 0:
good bad wrong old right cold fried dry little free extra sure hard terrible small slow sweet big new
Topic 1:
delicious fresh good friendly best fantastic tasty super nice sweet french cheese little excellent worth attentive garlic quick clean
Topic 2:
mexican red second horrible white vegetarian indian prime wine vegan limited gross possible exceptional wide giant lovely 10 oh
Topic 3:
decent italian reasonable fish flavorful entire frozen sushi rare pricey total simple dish complimentary polite pleased est cooked recent
Topic 4:
favorite new open fine warm real late available long poor local disappointed loud decent old single better original live
Topic 5:
great good amazing chinese delicious rude creamy wait fair right casual olive ridiculous wish biggest fried favourite strange absolute
Topic 6:
ok green authentic short worst best asian outside korean japanese black sour standard raw thai las second like weird
Topic 7:
best excellent awesome perfect wonderful delicious am

In [101]:
display_topics(adj_nmf_model, tfidf_adjs_feature_names, 20)

Topic 0:
good overall expensive reasonable fried solid hard ok cheap fish dry cheese garlic high slow cool real loud chicken
Topic 1:
great local wonderful reasonable live cool attentive outstanding overall atmosphere fun wine casual outdoor fabulous beautiful unique comfortable affordable
Topic 2:
small large expensive high tiny local worth hungry overall hard disappointed main limited extra smaller cozy japanese bigger larger
Topic 3:
best long mexican italian las indian worth chinese incredible authentic outstanding real fried korean fish average french entire phenomenal
Topic 4:
nice helpful attentive overall cool outside quiet pretty outdoor comfortable pleasant open beautiful warm local casual main able loud
Topic 5:
delicious authentic attentive wonderful beautiful flavorful healthy unique french glad worth vegetarian garlic yummy reasonable generous affordable warm fried
Topic 6:
friendly helpful attentive fast quick reasonable efficient knowledgeable prompt wait local professi

In [102]:
display_topics(adj_nmf_model_20, tfidf_adjs_feature_names, 20)

Topic 0:
good overall reasonable decent mexican solid cheap expensive fried cheese quick slow fast hard fish clean ok garlic chinese
Topic 1:
great fantastic wonderful local reasonable attentive live perfect cool outstanding atmosphere wine fun overall quick casual outdoor beautiful fabulous
Topic 2:
small big special different large tasty busy huge free decent sweet long ok overall cheese high old perfect right
Topic 3:
best mexican long italian fantastic las perfect chinese indian authentic worth incredible outstanding wonderful real fried french cheese korean
Topic 4:
nice clean helpful attentive overall cool outside quiet pretty comfortable outdoor pleasant beautiful open casual warm modern reasonable local
Topic 5:
delicious perfect wonderful sweet fantastic attentive beautiful authentic huge flavorful french cheese unique garlic healthy glad fried green red
Topic 6:
friendly clean helpful quick attentive fast reasonable efficient knowledgeable prompt wait local comfortable tasty 

In [103]:
display_topics(adj_nmf_model_10, tfidf_adjs_feature_names, 20)

Topic 0:
good overall decent reasonable tasty quick cheap fried expensive mexican solid cheese ok busy fish fast chinese hard clean
Topic 1:
great fantastic happy amazing wonderful local favorite perfect reasonable attentive cool live outstanding huge quick wine fun atmosphere new
Topic 2:
little bad small better sure new big hot happy different special busy large long decent old sweet huge favorite
Topic 3:
best amazing mexican favorite fantastic perfect long italian chinese las authentic indian worth incredible wonderful outstanding real fried sure
Topic 4:
nice clean super helpful tasty attentive overall cool pretty outside free quiet comfortable decent outdoor big open pleasant beautiful
Topic 5:
delicious amazing perfect favorite wonderful super sweet fantastic huge attentive beautiful authentic cheese french flavorful green fried unique garlic
Topic 6:
friendly clean helpful quick super attentive fast reasonable favorite efficient amazing tasty knowledgeable free fantastic local 

In [36]:
with open("nouns_tfidf_model.pkl", "w") as f:
    pickle.dump(tfidf_vectorizer_nouns, f)

In [65]:
with open("nouns_tf_model.pkl", "w") as f:
    pickle.dump(count_vectorizer_nouns, f)

In [66]:
with open("adjs_tfidf_model.pkl", "w") as f:
    pickle.dump(tfidf_vectorizer_adjs, f)

In [67]:
with open("adjs_tf_model.pkl", "w") as f:
    pickle.dump(count_vectorizer_adjs, f)

In [74]:
with open("adj_nmf_model_10.pkl", "w") as f:
    pickle.dump(adj_nmf_model_10, f)

In [75]:
with open("adj_nmf_model_20.pkl", "w") as f:
    pickle.dump(adj_nmf_model_20, f)

In [11]:
with open("adj_nmf_model_30.pkl", "w") as f:
    pickle.dump(adj_nmf_model, f)

In [77]:
with open("adj_lda_model_10.pkl", "w") as f:
    pickle.dump(adj_lda_model_10, f)

In [78]:
with open("adj_lda_model_20.pkl", "w") as f:
    pickle.dump(adj_lda_model_20, f)

In [79]:
with open("adj_lda_model_30.pkl", "w") as f:
    pickle.dump(adj_lda_model, f)

In [30]:
with open("noun_nmf_model_10.pkl", "w") as f:
    pickle.dump(noun_nmf_model_10, f)

In [31]:
with open("noun_nmf_model_20.pkl", "w") as f:
    pickle.dump(noun_nmf_model_20, f)

In [32]:
with open("noun_nmf_model_30.pkl", "w") as f:
    pickle.dump(noun_nmf_model, f)

In [33]:
with open("noun_lda_model_10.pkl", "w") as f:
    pickle.dump(noun_lda_model_10, f)

In [34]:
with open("noun_lda_model_20.pkl", "w") as f:
    pickle.dump(noun_lda_model_20, f)

In [35]:
with open("noun_lda_model_30.pkl", "w") as f:
    pickle.dump(noun_lda_model, f)